# Cosmos Db with container app
This template sets up a containerized application with a cosmos db connection. Please ensure an Container Environment is created before proceeding with the deployment, see [/scenario-container-app-environment.ipynb](../container-app-environment/scenario-container-app-environment.ipynb).

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- Container app environment ([See scenario-container-app-environment.ipynb](../container-app-environment/scenario-container-app-environment.ipynb))



## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [ ]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=northeurope

2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "cosmos-db-deployment"
template_file = "main.bicep"
parameters_file = "main.bicepparam"

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

Subscription ID: 7ba9e6b6-7591-4828-aaa5-c4656cb871c3
Resource Group: cosmos-container-app
Location: northeurope


4. Create an Azure Resource group:

In [ ]:
!az login
!az account set --subscription  {subscription_id}
!az group create --name {resource_group} --location {location}

## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [ ]:
%%writefile {parameters_file}

using 'main.bicep'

var databaseName = 'testdb'

param cosmosDbConfig = {
  accountName: take('csmdb${uniqueString(databaseName)}', 15)
  database: {
    name: databaseName
  }
  containers: [
    {
      name: 'movies'
      partitionKey: {
        kind: 'Hash'
        paths: [
          '/id'
        ]
      }
    }
    {
      name: 'movies_leases'
      partitionKey: {
        kind: 'Hash'
        paths: [
          '/id'
        ]
      }
    }
    {
      name: 'movies_by_genre'
      partitionKey: {
        kind: 'Hash'
        paths: [
          '/genre'
        ]
      }
    }
  ]
}

param containerAppConfig = {
  environment: {
    name: 'my-container-app-env'
    resourceGroup: 'container-apps'
  }
  name: 'my-cosmos-container-app'
  ingress: {
    allowInsecure: false
    clientCertificateMode: 'ignore'
    external: true
    targetPort: 80
    transport: 'auto'
    traffic: [
      {
        latestRevision: true
        weight: 100
      }
    ]
  }
  containers: [
    {
      registry: 'mcr.microsoft.com'
      image: 'azuredocs/containerapps-helloworld:latest'
      name: 'hello-world-container'
      resources: {
        cpu: '0.25'
        memory: '0.5Gi'
      }
      env: [
        {
          name: 'TEST_VAR1'
          value: 'hello world'
        }
      ]
    }
  ]
  scale: {
    minReplicas: 1
    maxReplicas: 2
    rules: [
      {
        name: 'http-requests'
        http: {
          metadata: {
            concurrentRequests: '10'
          }
        }
      }
    ]
  }
}

2. Deploy the template:

In [ ]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name} 